In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from requests import get


In [ ]:
base_url = 'https://eur-lex.europa.eu'
response = get('https://eur-lex.europa.eu/browse/directories/legislation.html')
html_soup = BeautifulSoup(response.text,'html.parser')
Document_container = html_soup.find_all('ul',class_='browseTree', id="tree" )
all_documents_to_parse = Document_container[0].find_all('li', class_ = '')
all_documents_to_parse.extend(Document_container[0].find_all('li', class_ = 'leaf'))
crawled_data = pd.DataFrame()
final_celex = []
summary = []
text = []
celex_list = []
data = pd.DataFrame()
passed_href = []
not_passed_href = []

#for docs in all_documents_to_parse:
for i in range(11,12):  
  docs = all_documents_to_parse[i]
  links = len(docs.find_all('a'))
  if links> 1:
    next_page_href= docs.find_all('a')[1]
  else:
    next_page_href= docs.find_all('a')[0]

  #print(next_page_href)                                    #uncomment later
  href = next_page_href.get('href')
  #print(href)                                              #uncomment later
  pages_url = ['https://eur-lex.europa.eu/'+href[8:]]
  
  print(docs)
    
  for url in pages_url:
    print("page url : {}".format(url))
    page = get(url)
    html_soup_inner_page = BeautifulSoup(page.text,'html.parser')
    html_soup_xmlsns = html_soup_inner_page.find_all('div',xmlns='http://www.w3.org/1999/xhtml' ) #has the search results


    ########################################## CELEX ###############################################
    #In the search results page
    for documents in html_soup_xmlsns:
        doc_collapse = documents.find_all('div', class_ = 'CollapsePanel-sm')[0]
        doc_collapse_searchdatacollapse = doc_collapse.find_all('div', class_ = 'SearchResultData collapse in')[0]
        doc_collapse_searchdatacollapse_row = doc_collapse_searchdatacollapse.find_all('div', class_ = 'row')[0]
        doc_collapse_searchdatacollapse_row_colsm6 = doc_collapse_searchdatacollapse_row.find_all('div', class_ = 'col-sm-6')[0]
        celex = doc_collapse_searchdatacollapse_row_colsm6.find_all('dd')[0].text
        print(celex)
        celex_list.append(celex)

    ############################################ TEXT DATA ################################################

    for i in range(0,len(html_soup_xmlsns)):
      html_soup_xmlsns_h2 = html_soup_xmlsns[i].find_all('h2')
      html_soup_xmlns_h2_a = html_soup_xmlsns_h2[0].find_all('a')
      final_href = html_soup_xmlns_h2_a[0].get('href')
      inner_page_url = 'https://eur-lex.europa.eu/'+final_href[2:] #navigate to selected page
      print("inner page url : {}".format(inner_page_url))
      print(inner_page_url)
      inner_page = get(inner_page_url)
      html_soup_final_page = BeautifulSoup(inner_page.text,'html.parser') 

      doc_text = html_soup_final_page.find_all('div', class_= 'panel-collapse collapse in', id = 'PP4Contents') #text content extraction
      doc_summary = html_soup_final_page.find_all('div', class_ = 'panel-body')


      html_soup_final_page_celex = html_soup_final_page.find_all('div', class_ = 'PageTitle') #to get celex number
      if len(html_soup_final_page_celex) > 0:
        html_soup_final_page_celex2 = html_soup_final_page_celex[0].find_all('div', class_ = 'col-xs-8 col-sm-4 col-md-3 col-md-push-3')
        if len(html_soup_final_page_celex2) > 0:
          final_celex = html_soup_final_page_celex2[0].find_all('h1') #celex number present here



            # save data in crawled dataframe
      row = {"celex" : final_celex[0].text if (len(final_celex)>0) else "", "text" : doc_text[0].text if (len(doc_text)>0) else "", "summary": doc_summary[0].text if (len(doc_summary)>0) else ""}
      #print(row)
      crawled_data =  crawled_data.append(row,ignore_index = True)







    try:
      next_page_info = html_soup_inner_page.find_all('form', id = 'pagingForm')[0]
      passed_href.append(url)
    except:
      print(html_soup_inner_page) 
      not_passed_href.append(url) 
    next_page_data= next_page_info.find_all('a', title = 'Next Page')
    if(len(next_page_data) >0):
      href = next_page_data[0].get('href')
      pages_url.append('https://eur-lex.europa.eu'+href[1:])


  for url in not_passed_href:
    print("page url : {}".format(url))
    page = get(url)
    html_soup_inner_page = BeautifulSoup(page.text,'html.parser')
    html_soup_xmlsns = html_soup_inner_page.find_all('div',xmlns='http://www.w3.org/1999/xhtml' )




<li class="">
	12 Energy</a>
<a class="gotoResultLink" href="./../../search.html?type=named&amp;name=browse-by:legislation-in-force&amp;CC_1_CODED=12&amp;displayProfile=allRelAllConsDocProfile" id="search_12" onclick="showHourglass();" title="(12 Energy, number of acts: 472)">472</a>
</li>
page url : https://eur-lex.europa.eu/search.html?type=named&name=browse-by:legislation-in-force&CC_1_CODED=12&displayProfile=allRelAllConsDocProfile
32020R0740
32019L0944
32019R0941
32019R0943
32019R0942
32018R1999
32017R1938
32017R1369
32017D0684
32016R1952
inner page url : https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:32020R0740&qid=1637824525604&rid=1
https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:32020R0740&qid=1637824525604&rid=1
inner page url : https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:32019L0944&qid=1637824525604&rid=2
https://eur-lex.europa.eu/legal-content/AUTO/?uri=CELEX:32019L0944&qid=1637824525604&rid=2
inner page url : https://eur-lex.europa.eu/legal-con

In [ ]:
crawled_data.to_pickle('data_crawled_final.pkl') 

In [ ]:
ls

data_crawled_final.pkl  sample_data/


In [ ]:
data

In [ ]:
crawled_data.celex

0       
1       
2       
3       
4       
      ..
467     
468     
469     
470     
471     
Name: celex, Length: 472, dtype: object

**Topic pickle file (not running yet)**

In [ ]:
topics = []
for celex in data['celex']:
  document_celex_number = celex.split()
  print(" processing for celex {}".format(document_celex_number))
  page = get('http://eur-lex.europa.eu/LexUriServ/LexUriServ.do?uri=CELEX:{}:EN:NOT'.format(document_celex_number))
  html_soup_inner_page = BeautifulSoup(page.text,'html.parser')
  try:
    html_soup_data = html_soup_inner_page.find_all('div',id = 'PPClass_Contents', class_='panel-collapse collapse in')[0]
    html_soup_data_metadata = html_soup_data.find_all('dl', class_='NMetadata')[0]
    html_soup_data_dt = html_soup_data_metadata.find_all('dt')
    index_for_subject = [item.text for item in html_soup_data_dt].index("Subject matter: ")
    html_soup_data_dd = html_soup_data_metadata.find_all('dd')[index_for_subject]
    html_soup_data_dd_li = html_soup_data_dd.find_all('li')
    topics.append({'celex':document_celex_number, "subjects" :[item.text.strip() for item in html_soup_data_dd_li]})
  except:
    topics.append({'celex':document_celex_number, "subjects" :[]})
  
print(topics)